# Анализ сети фитнес-центров. Подготовка плана действий по удержанию клиентов.

### <a name="7."></a>[Содержание](#1.)

* [1. Описание проекта](#1.)
* [2. Загрузка данных и подготовка их к анализу](#2.)
* [2. Исследовательский анализ данных (EDA)](#3.)
* [3. Создание модели прогнозирования оттока пользователей](#4.)
* [4. Кластеризация пользователей](#5.)
* [5. Выводы](#6.)


### 1. Описание проекта

Сеть фитнес-центров «Культурист-датасаентист» разрабатывает стратегию взаимодействия с пользователями на основе аналитических данных.
Распространённая проблема фитнес-клубов и других сервисов — отток клиентов. Как понять, что пользователь больше не с вами? Можно записать в отток тех, кто попросил закрыть договор или удалил аккаунт. Однако клиенты не всегда уходят демонстративно: чаще перестают пользоваться сервисом тихо.

Индикаторы оттока зависят от специфики отрасли. Когда пользователь редко, но стабильно закупается в интернет-магазине — не похоже, что он «отвалился». А вот если две недели не заходит на канал с ежедневно обновляемым контентом, дела плохи: подписчик заскучал и, кажется, оставил вас.

Для фитнес-центра можно считать, что клиент попал в отток, если за последний месяц ни разу не посетил спортзал. Конечно, не исключено, что он уехал на Бали и по приезде обязательно продолжит ходить на фитнес. Однако чаще бывает наоборот. Если клиент начал новую жизнь с понедельника, немного походил в спортзал, а потом пропал — скорее всего, он не вернётся.

Чтобы бороться с оттоком, отдел по работе с клиентами «Культуриста-датасаентиста» перевёл в электронный вид множество анкет пользователей. Ваша задача — провести анализ и подготовить план действий по удержанию клиентов.

А именно:
- научиться прогнозировать вероятность оттока (на уровне следующего месяца) для каждого клиента;
- сформировать типичные портреты пользователей: выделить несколько наиболее ярких групп и охарактеризовать их основные свойства;
- проанализировать основные признаки, наиболее сильно влияющие на отток;
- сформулировать основные выводы и разработать рекомендации по повышению качества работы с клиентами:
- 1) выделить целевые группы клиентов;
- 2) предложить меры по снижению оттока;
- 3) определить другие особенности взаимодействия с клиентами.

### 2. <a name="2."></a>Загрузка данных и подготовка их к анализу

Заказчик подготовил данные, которые содержат информацию на месяц до оттока и факт оттока на определённый месяц. Набор данных включает следующие поля:

- `'Churn'` — факт оттока в текущем месяце;
- Текущие поля в датасете:
    - Данные пользователя за предыдущий до проверки факта оттока месяц:
        - `'gender'` — пол
        - `'Near_Location'` — проживание или работа в районе, где находится фитнес-центр
        - `'Partner'` — сотрудник компании-партнёра клуба (сотрудничество с компаниями, чьи сотрудники могут получать скидки на абонемент — в таком случае фитнес-центр хранит информацию о работодателе клиента)
        - `Promo_friends` — факт первоначальной записи в рамках акции «приведи друга» (использовал промо-код от знакомого при оплате первого абонемента)
        - `'Phone'` — наличие контактного телефона
        - `'Age'` — возраст
        - `'Lifetime'` — время с момента первого обращения в фитнес-центр (в месяцах)
    - Информация на основе журнала посещений, покупок и информация о текущем статусе абонемента клиента
        - `'Contract_period'` — длительность текущего действующего абонемента (месяц, 3 месяца, 6 месяцев, год)
        - `'Month_to_end_contract'` — срок до окончания текущего действующего абонемента (в месяцах)
        - `'Group_visits'` — факт посещения групповых занятий
        - `'Avg_class_frequency_total'` — средняя частота посещений в неделю за все время с начала действия абонемента
        - `'Avg_class_frequency_current_month'` — средняя частота посещений в неделю за предыдущий месяц
        - `'Avg_additional_charges_total'` — суммарная выручка от других услуг фитнес-центра: кафе, спорт-товары, косметический и массажный салон

In [ ]:
#импорт библиотек
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import numpy as np
pd.options.display.max_columns = 500 
pd.set_option('display.max_colwidth', 200)
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.cluster import KMeans

In [ ]:
data = pd.read_csv('/datasets/gym_churn.csv')

In [ ]:
#создадим функцию для вывода информации о датасете
def info(data):
    display(data.head(10))
    print('Информация о датафрейме')
    print(data.info())
    print()
    print('Первые 10 строк')
    print()
    for i in data.columns:
        print('Число пропущенных значений в столбце ', i, ' = ', data[i].isnull().sum())
    print('Число дубликатов в датафрейме', data.duplicated().sum())

In [ ]:
info(data)

В датасете 4000 наблюдений и 14 полей — 13 признаков и целевая переменная. Все признаки численные, пропусков нет.

В датасете содержся бинарные поля, признаки которых принимают только значения 0 или 1: 'gender', 'Near_Location', 'Partner', 'Promo_friends', 'Phone', Group_visits. Остальные числовые поля: 'Contract_period', 'Age', 'Avg_additional_charges_total', 'Month_to_end_contract', 'Lifetime','Avg_class_frequency_total', 'Avg_class_frequency_current_month', 'Churn'.

### 3. <a name="3."></a>Исследовательский анализ данных (EDA)
[к содержанию](#1.)

Сохраним в data1 признаки, не относящиеся к бинарным полям.

In [ ]:
data1=data[['Contract_period', 'Age', 'Avg_additional_charges_total', 'Month_to_end_contract', 'Lifetime',
      'Avg_class_frequency_total', 'Avg_class_frequency_current_month', 'Churn']]

Посмотрим на статистику признаков в двух группах — тех, кто ушел в отток и тех, кто остался.

In [ ]:
data1[data1['Churn']==0].describe()

In [ ]:
data1[data1['Churn']==1].describe()

Для клиентов, которые не ушли, в сравнении с теми, кто попал в отток, характерно следующее:
- большая длительность абонемента,
- больший средний возраст (30 против 27),
- большая средняя частота посещений,
- большее пользование услугами фитнес-центра: кафе, спорт-товары, косметический и массажный салон

In [ ]:
fig = go.Figure()
for i in data1.columns: 
    y = data[i]
    fig.add_trace(go.Box(y=y, name=i, boxmean=True))

fig.update_xaxes(title='Тип признака')
fig.update_layout(template="plotly_white", title="Распределение наблюдений по признакам", 
                  showlegend = False, autosize=False, width=950, height=600)
fig.show()

Для лучшей визуализации посмотрим на распределение признаков без `Avg_additional_charges_total'`.

In [ ]:
fig = go.Figure()
for i in data1.drop('Avg_additional_charges_total', axis = 1).columns: 
    y = data[i]
    fig.add_trace(go.Box(y=y, name=i, boxmean=True))

fig.update_xaxes(title='Тип признака')
fig.update_layout(template="plotly_white", title="Распределение наблюдений по признакам", 
                  showlegend = False, autosize=False, width=950, height=600)
fig.show()

Редкие значения в небольшом количестве наблюдается почти у всех признаков, кроме `'Contract_period'` и `'Month_to_end_contract'`.

Построим гистограммы признаков для тех, кто ушёл (отток) и тех, кто остался (не попали в отток):

In [ ]:
for col in data1.drop('Churn', axis = 1).columns:
    x1 = data1[data1['Churn']==0][col]
    x2 = data1[data1['Churn']==1][col]
    x3 = data1[col]
# plot
    fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharey=True, sharex=True)
    fig.suptitle('Данные по столбцу: ' + col)
    sns.distplot(x1 , color="dodgerblue", ax=axes[0], label='No Churn')
    sns.distplot(x2 , color="red", ax=axes[0], axlabel='Churn + No Churn', label = 'Churn')
    sns.distplot(x3 , color="green", ax=axes[1], axlabel='All data')
    fig.legend(bbox_to_anchor=(0.066, 0.9), loc='lower left', ncol=2, borderaxespad=0.)
    axes[0].set(ylabel='Density')
    axes[0].grid(axis='y', color='0.95')
    axes[1].grid(axis='y', color='0.95')
    plt.show()

Величина признаков у тех, кто остался (не ушел в отток) имеет большие значения. Наибольшее отличие наблюдается у признака `'Avg_class_frequency_current_month'` (средняя частота посещений в неделю за предыдущий месяц). В следующем шаге применим стандартизацию данных для приведения кнормальному распределению.

Построим и отрисуем матрицу корреляций:

In [ ]:
cm = data.corr() #вычисляем матрицу корреляций

#нарисуем тепловую карту с подписями для матрицы корреляций
plt.figure(figsize=(13, 9))
sns.heatmap(cm, annot=True, fmt='.1f', linewidths=1, linecolor='gray')

Корреляция призаков с целевой переменной невысока. Больше всего с целевой переменной коррелируют признаки:
* `'Contract_period'` — длительность текущего действующего абонемента,
* `'Age'` — возраст,
* `'Month_to_end_contract'` — срок до окончания текущего действующего абонемента,
* `'Lifetime'` — время с момента первого обращения в фитнес-центр,
* `'Avg_class_frequency_current_month'` — средняя частота посещений в неделю.

Сильная парная корреляция (0.5) у признаков: `'Partner'` (сотрудник компании-партнёра клуба) - `'Promo_friends'` (факт первоначальной записи в рамках акции «приведи друга»).

### 4. <a name="4."></a>Создание модели прогнозирования оттока пользователей
[к содержанию](#1.)

Сохраним признаки в матрице X и целевую переменную в y:

In [ ]:
X = data.drop('Churn', axis = 1)
y = data['Churn']

Разделим наш датасет на части: 
- 60% -обучающая, 
- 20% - валидационная
- 20% - тестовая.

In [ ]:
#получим обучающую выборку (X_train, y_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, train_size=0.6)

In [ ]:
#получим валидационную (X_val, y_val) и тестовую выборки (X_train, y_train)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test ,test_size = 0.5,train_size =0.5)

Стандартизируем выборки:

In [ ]:
# создадим объект класса StandardScaler и применим его к обучающей выборке
scaler = StandardScaler()

In [ ]:
#обучаем scaler и одновременно трансформируем матрицу для обучающей выборки
X_train_st = scaler.fit_transform(X_train)

In [ ]:
# применяем стандартизацию к матрице признаков для валидационной и тестовой выборки
X_val_st = scaler.transform(X_val)
X_test_st = scaler.transform(X_test)

Обучим модель на train-выборке двумя способами:
- логистической регрессией - sklearn.linear_model.LogisticRegression(),
- случайным лесом - sklearn.ensemble.RandomForestClassifier().

In [ ]:
# зададим алгоритм для модели
lr_model = LogisticRegression()

In [ ]:
# обучим модель
lr_model.fit(X_train_st, y_train)


In [ ]:
#рассчитаем прогноз класса после обучения
lr_predictions = lr_model.predict(X_val_st)

# получим вероятность принадлежности объекта к первому или второму классу
lr_probabilities = lr_model.predict_proba(X_val_st)[:,1]


In [ ]:
# получим метрики для полученного прогноза
print('Метрики логистической регрессии sklearn.linear_model.LogisticRegression():')
print('Accuracy: {:.2f}'.format(accuracy_score (y_val, lr_predictions)))
print('Precision: {:.2f}'.format(precision_score (y_val, lr_predictions)))
print('Recall: {:.2f}'.format(recall_score (y_val, lr_predictions)))


Модель логистической регрессии правильно прогнозирует 93%

**Выполним перебор следующих гиперпараметров случайного леса:**
- n_estimators, 
- max_depth
- min_samples_leaf

In [ ]:
n_estimators = [300,500,700]
max_depth = [2,3,7,11,15]
min_samples_leaf = [2,3,4,5,6,7]


In [ ]:
for i in n_estimators:
    rfc_model = RandomForestClassifier(n_estimators = i)
    rfc_model.fit(X_train_st, y_train)
    rfc_predictions = rfc_model.predict(X_val_st)
    rfc_probabilities = rfc_model.predict_proba(X_val_st)[:,1]
    print('Метрики случайного леса n_estimators =', i)
    print('Accuracy: {:.2f}'.format(accuracy_score (y_val, rfc_predictions)))
    print('Precision: {:.2f}'.format(precision_score (y_val, rfc_predictions)))
    print('Recall: {:.2f}'.format(recall_score (y_val, rfc_predictions)))
print()
for i in max_depth:
    rfc_model = RandomForestClassifier(max_depth = i)
    rfc_model.fit(X_train_st, y_train)
    rfc_predictions = rfc_model.predict(X_val_st)
    rfc_probabilities = rfc_model.predict_proba(X_val_st)[:,1]
    print('Метрики случайного леса max_depth =', i)
    print('Accuracy: {:.2f}'.format(accuracy_score (y_val, rfc_predictions)))
    print('Precision: {:.2f}'.format(precision_score (y_val, rfc_predictions)))
    print('Recall: {:.2f}'.format(recall_score (y_val, rfc_predictions)))
print()
for i in min_samples_leaf:
    rfc_model = RandomForestClassifier(min_samples_leaf = i)
    rfc_model.fit(X_train_st, y_train)
    rfc_predictions = rfc_model.predict(X_val_st)
    rfc_probabilities = rfc_model.predict_proba(X_val_st)[:,1]
    print('Метрики случайного леса min_samples_leaf =', i)
    print('Accuracy: {:.2f}'.format(accuracy_score (y_val, rfc_predictions)))
    print('Precision: {:.2f}'.format(precision_score (y_val, rfc_predictions)))
    print('Recall: {:.2f}'.format(recall_score (y_val, rfc_predictions)))


Гиперпараметр n_estimators = 500 обеспечивает лучший результат:
- Accuracy: 0.92
- Precision: 0.87
- Recall: 0.84

Метрики логистической регрессии sklearn.linear_model.LogisticRegression():
- Accuracy: 0.91
- Precision: 0.88
- Recall: 0.80

Используем модель случайного леса с гипераметром n_estimators = 300 для работы с тестовой выборкой:

In [ ]:
rfc_model = RandomForestClassifier(n_estimators = 500)
rfc_model.fit(X_train_st, y_train)
rfc_predictions = rfc_model.predict(X_test_st)
rfc_probabilities = rfc_model.predict_proba(X_test_st)[:,1]
print('Метрики случайного леса')
print('Accuracy: {:.2f}'.format(accuracy_score (y_test, rfc_predictions)))
print('Precision: {:.2f}'.format(precision_score (y_test, rfc_predictions)))
print('Recall: {:.2f}'.format(recall_score (y_test, rfc_predictions)))

**Точность прогнозов - 91%. Остальные метрики также показывают хорошие цифры.**

### 5. <a name="5."></a>Кластеризация пользователей
[к содержанию](#1.)

In [ ]:
# стандартизируем данные
sc = StandardScaler()
x_sc = sc.fit_transform(data.drop('Churn', axis = 1))

Построим матрицу расстояний функцией linkage() и нарисуем дендрограмму:

In [ ]:
Z = linkage(x_sc, 'ward')
fig = plt.figure(figsize=(25, 10))
dn = dendrogram(Z)
plt.show()

На графике мы видим 4 разных цвета, при этом фиолетовая область самая обширная - на ней выделяются 3 кластера. Остановимся на числе кластеров равным 5.

Обучим модель кластеризации на основании алгоритма K-Means с числом кластеров n=5:

In [ ]:
# задаём модель k_means, укажем параметр random_state=0, чтобы сделать случайность детерминированной
km = KMeans(n_clusters = 5, random_state=0)

In [ ]:
# прогнозируем кластеры для клиентов
labels = km.fit_predict(x_sc)

In [ ]:
# сохраняем метки кластера в поле нашего датасета
data['cluster'] = labels

In [ ]:
# выводим статистику по средним значениям наших признаков по кластеру
data.groupby(['cluster']).mean()

In [ ]:
for col in data.drop('cluster', axis = 1).columns:
    data.groupby(['cluster'])[col].mean().sort_values().plot.bar(grid=True, color="dodgerblue", 
                                        figsize=(5, 3))
    plt.title(col, fontsize=17)
    plt.show()

Наиболее якркие кластеры №0 и №4 (имеющие наибольшие средние величины признаков). Далее приведены характеристики кластеров.\
Кластер #0:
- наибольшая доля клиентов, относящиеся к сотрудникам компании-партнёра клуба,
- наибольшая доля клиентов, воспользовавшихся акцией «приведи друга»,
- наибольшая длительность действующего абонемента,
- наибольшая доля клиентов, посещающих групповые занятия,
- высокая средняя частота посещений,
- клиенты активно пользуются другими услугами фитнес-центра,
- низкий отток.

Кластер #1 
- около половины составляют клиенты, относящиеся к сотрудникам компании-партнёра клуба,
- около трети воспользовались ацией «приведи друга»,
- абонементы больщой продолжительностью (в среднем 7,78мес)
- по другим признакам характерны средние значения, относительно других кластеров.

Кластер #2 
- клиенты, не проживают/работают рядом с фитнес-центром,
- малая доля клиентов, воспользовавшихся акцией «приведи друга»,
- абонементы небольшой продолжительностью (в среднем 2,35мес),
- мало посещают групповые занятия,
- низкая частота посещений,
- высокий отток.

Кластер #3 
- наименьшая доля клиентов, относящиеся к сотрудникам компании-партнёра клуба,
- минимальная средняя продолжительность абонемента,
- небольшая в групповых занятиях,
- минимальная частота посещений
- максимальный отток.


Кластер #4:
- наибольшая доля клиентов, активно пользующихся другими услугами фитнес-центра,
- наибольшая средняя частота посещений,
- низкий отток.


Построим гистограммы признаков для кластеров:

In [ ]:
for col in data1.drop(['Churn'], axis = 1).columns:
    x1 = data[data['cluster']==0][col]
    x2 = data[data['cluster']==1][col]
    x3 = data[data['cluster']==2][col]
    x4 = data[data['cluster']==3][col]
    x5 = data[data['cluster']==4][col]
# plot
    fig, axes = plt.subplots(1, figsize=(15, 5), sharey=True, sharex=True)
    fig.suptitle('Данные по столбцу: ' + col)
    sns.distplot(x1 , color="dodgerblue", ax=axes, label='cluster 0')
    sns.distplot(x2 , color="red", ax=axes, label = 'cluster 1')
    sns.distplot(x3 , color="green", ax=axes, label = 'cluster 2')
    sns.distplot(x4 , color="yellow", ax=axes, label = 'cluster 3')
    sns.distplot(x5 , color="orange", ax=axes, label = 'cluster 4')
    fig.legend(bbox_to_anchor=(0.066, 0.9), loc='lower left', ncol=2, borderaxespad=0.)
    axes.grid(axis='y', color='0.95')
    axes.grid(axis='y', color='0.95')
    plt.show()

4 и 0 кластеры выделяются на фоне других большей величиной признаков.

Рассчитаем долю оттока для каждого кластера:

In [ ]:
data.groupby('cluster')['Churn'].mean().sort_values()

Минимальный отток у 0 и 4 кластеров. У 1 кластера отток составляет  26%, 2 - 44%, наибольший отток у 3 кластера (50%).

### 6. <a name="6."></a>Выводы
[к содержанию](#1.)

- В ходе исследования выделено пять групп клиентов,
- К целевым клиентам относятся люди в возрасте около 30 лет (среди них наименьшая доля оттока),
- Основные целевые групп две. Первую группу (кластер 0) характеризуют следующие признаки:
    - наибольшая доля клиентов, относящиеся к сотрудникам компании-партнёра клуба,
    - наибольшая доля клиентов, воспользовавшихся акцией «приведи друга»,
    - наибольшая длительность действующего абонемента,
    - наибольшая доля клиентов, посещающих групповые занятия,
    - высокая средняя частота посещений,
    - клиенты, активно пользующихся другими услугами фитнес-центра,
    - низкий отток.
- Отличительные признаки второй целевой группы (кластер 4):
    - наибольшая доля клиентов, активно пользующихся другими услугами фитнес-центра,
    - наибольшая средняя частота посещений,
    - низкий отток.

Рекомендации.
- Для сокращения оттока в 3 группе, где наблюдается высокий отток и около 2 мес до окончания действия абонемента -  сделать спец. предложения по продлению членства в клубе, вовлекать в групповые программы, например, предлагать бесплатные первые посещения, бесплатные посещения групповых программ для друзей и т.п.
- Для самой ценной группы (кластер 0)- поддержка лояльности, работа в соц. сетях, расширение направления групповых программ.
